# TP 1 - Análisis Exploratorio de Datos
## Formula 1 World Championship (1950 - 2023)
El dataset elegido contiene toda la información acerca de las carreras de Fórmula 1, pilotos, constructores, clasificación, circuitos, tiempos de vuelta, paradas en boxes y campeonatos desde 1950 hasta la última temporada de 2023.
El mismo cuenta con 14 archivos csv, de los cuales decidimos usar solamente algunos. Esto se debe a que vamos a tratar de predecir si la posición final del piloto va a estar o no en el podio, por lo que pensamos hacer la predicción en el momento que está finalizada la clasificación, con los datos disponibles hasta ese momento.

### 1) Listado de variables y selección
__CSVs y variables de entrada:__
Todos estos archivos .csv de los que está compuesto el set de datos tienen columnas con Ids, las cuales utilizaremos a nuestra conveniencia para realizar ‘joins’ entre las distintas tablas. Antes de entrenar, todas las columnas que hacen referencia a Ids serán eliminadas para que no se genere overfitting a partir de estas.

#### Variables de entrada

__Races.csv__

_Variables a usar_
- date: indica la fecha de la carrera.
- time: indica la hora en que inicia la carrera
- year: año en que se realiza la carrera

_Variables no usadas_

- name: nombre de la carrera que se va a correr, utilizaremos referencias al circuito.
- url: url que brinda más información sobre la carrera, no nos parece que influya en lo que vamos a predecir.
- fp1_date: fecha de inicio de la práctica libre número uno, no nos parece que influya en lo que vamos a predecir.
- fp1_time: hora de inicio de la práctica libre número uno, no nos parece que influya en lo que vamos a predecir.
- fp2_date: fecha de inicio de la práctica libre número dos, no nos parece que influya en lo que vamos a predecir.
- fp2_time: hora de inicio de la práctica libre número dos, no nos parece que influya en lo que vamos a predecir.
- fp3_date: fecha de inicio de la práctica libre número tres, no nos parece que influya en lo que vamos a predecir.
- fp3_time: hora de inicio de la práctica libre número tres, no nos parece que influya en lo que vamos a predecir.
- quali_date: fecha de inicio de la sesión de clasificación, no nos parece que influya en lo que vamos a predecir.
- quali_time: hora de inicio de la sesión de clasificación, no nos parece que influya en lo que vamos a predecir.
- sprint_date: fecha de inicio de la carrera sprint, no será usado ya que nos centraremos en la clasificación debido a que no todas las carreras cuentan con sprint.
- sprint_time: hora de inicio de la carrera sprint, no será usado ya que nos centraremos en la clasificación debido a que no todas las carreras cuentan con sprint.

__Circuits.csv__

_Variables a usar_
- circuitRef: variable de texto que indica el nombre de referencia del circuito.

_Variables no usadas_

- name: nombre del circuito, utilizaremos circuitRef.
- location: localidad donde se disputa la carrera, no nos parece que influya en lo que vamos a predecir.
- country: país donde se disputa la carrera, no nos parece que influya en lo que vamos a predecir
- lat: latitud, no nos parece que influya en lo que vamos a predecir. 
- lng: longitud, no nos parece que influya en lo que vamos a predecir. Además, utilizaremos alt.
- url: url que brinda más información sobre el circuito, no nos parece que influya en lo que vamos a predecir.
- alt: variable numérica que indica la altitud del circuito, no nos parece que influya en lo que vamos a predecir.

__Results.csv__

_Variables a usar_
- grid: variable numérica que indica la posición de largada.

_Variables no usadas_

- number: número del auto, no nos parece que influya en lo que vamos a predecir.
- position: posición final del corredor en la carrera, no se usará ya que si el corredor no finaliza la carrera, esto se representa con un valor nulo.
- positionText: lo mismo que position pero en tipo texto.
- points: puntos obtenidos por el corredor al finalizar la carrera, este dato no lo tenemos al momento de predecir.
- laps: cantidad de vueltas realizadas por el corredor, este dato no lo tenemos al momento de predecir.
- time: tiempo total en la pista, este dato no lo tenemos al momento de predecir.
- milliseconds: tiempo total en pista en milisegundos, este dato no lo tenemos al momento de predecir. 
- fastestLap: número de vuelta de la vuelta más rápida del corredor, este dato no lo tenemos al momento de predecir.
- fastestLapTime: tiempo de vuelta, de la vuelta más rápida del corredor, este dato no lo tenemos al momento de predecir.
- fastestLapSpeed: velocidad máxima de la vuelta más rápida, este dato no lo tenemos al momento de predecir.
- statusId: id que hace referencia al status de finalización de la carrera, no será utilizado ya que no nos resulta relevante y es un dato que no tendremos al momento de hacer la predicción.

__Driver_standings.csv__

_Variables a usar_

- position (_ds_position_): variable numérica que indica la posición del corredor en el campeonato de corredores una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.
- wins (_ds_wins_): variable numérica que indica la cantidad de victorias en el campeonato una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.
- points (_ds_points_): variable numérica que indica la cantidad de puntos acumulados que tiene el corredor una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.

_Variables no usadas_

- positionText: variable de texto que indica la posición del corredor en el campeonato pero en tipo texto, usaremos la variable numérica.

__Constructor_standings.csv__

_Variables a usar_

- position (_cs_position_): variable numérica que indica la posición del equipo en el campeonato de constructores una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.
- wins (_cs_wins_): variable numérica que indica la cantidad de victorias del equipo en el campeonato de constructores una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.
- points (_cs_points_): variable numérica que indica la cantidad de puntos acumulados que tiene el equipo una vez finalizada una carrera, utilizaremos este dato pero de la carrera anterior.

_Variables no usadas_

- positionText: variable de texto que indica la posición del equipo en el campeonato de constructores pero en tipo texto, usaremos la variable numérica.

__Qualifying.csv__

_Variables a usar_

- position (_q_position_): variable numérica que indica la posición en la que finaliza el corredor la sesión de clasificación
- q1 (_q1_ms_): variable numérica que indica el tiempo en clasificación q1.
- q2 (_q2_ms_): variable numérica que indica el tiempo en clasificación q2.
- q3 (_q3_ms_): variable numérica que indica el tiempo en clasificación q3.

_Variables no usadas_

- number: número del auto, no nos parece que influya en lo que vamos a predecir.

___CSVs que no se utilizarán:___

* _constructor_results_: no se utilizará ninguna variable de esta tabla ya que son datos que no están disponibles al momento que vamos a realizar la predicción.
* _constructors_: no se utilizará ninguna variable de esta tabla ya que indican información general sobre los constructores y no creemos que sea relevante.
* _drivers_: no se utilizará ninguna variable de esta tabla ya que indican información general sobre los corredores y no creemos que sea relevante.
* _lap_times_: no se utilizará ninguna variable de esta tabla ya que son datos que no están disponibles al momento que vamos a realizar la predicción.
* _pit_stops_: no se utilizará ninguna variable de esta tabla ya que son datos que no están disponibles al momento que vamos a realizar la predicción.
* _seasons_: no se utilizará ninguna variable de esta tabla ya que indican información general sobre las temporadas  y no creemos que sea relevante.
* _sprint_results_: no se utilizará ninguna variable de esta tabla ya que indican tiempos de la carrera sprint y son datos que no estarán disponibles al momento de la predicción.
* _status_: mapea el id de los distintos estados en los que se puede finalizar una carrera. No lo utilizaremos ya que son datos que no están disponibles al momento de la predicción.

#### Variable de salida
___is_podium___: esta variable sale del csv de “results”, se va a transformar la columna “positionOrder” la cual hace referencia a la posición final del corredor en la carrera. “is_podium” es una variable _binaria_ que va a determinar si el piloto va a estar en el podio (primer, segundo y tercer puesto). Los valores que puede tener son _0_ que indica que el corredor no está en el podio y _1_ si el mismo se encuentra en el podio.

In [ ]:
# Importamos las dependencias que vamos a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import sklearn_pandas
from matplotlib import gridspec
from datetime import datetime

In [ ]:
# Quitamos el límite de columnas y filas que se muestran en los dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Importamos los csvs que vamos a utilizar
races = pd.read_csv('./csvs/races.csv')
circuits = pd.read_csv('./csvs/circuits.csv')
results = pd.read_csv('./csvs/results.csv')
driver_standings = pd.read_csv('./csvs/driver_standings.csv')
constructor_standings = pd.read_csv('./csvs/constructor_standings.csv')
qualifying = pd.read_csv('./csvs/qualifying.csv')

In [ ]:
# Eliminamos las columnas que no vamos a utilizar en cada dataset antes de hacer los merges
races.drop(['round', 'name', 'url', 'fp1_date', 'fp1_time','fp2_date', 'fp2_time','fp3_date',
            'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis=1, inplace=True)

circuits.drop(['name', 'location', 'country', 'lat', 'lng', 'url', 'alt'], axis=1, inplace=True)

results.drop(['number', 'position', 'positionText', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 
           'fastestLapTime', 'fastestLapSpeed', 'statusId', 'rank'], axis=1, inplace=True)

driver_standings.drop(['positionText'], axis=1, inplace=True)

constructor_standings.drop(['positionText'], axis=1, inplace=True)

qualifying.drop(['number'], axis=1, inplace=True)

In [ ]:
# Cambiamos nombres de columnas para identificarlos mejor
qualifying.rename(columns = {'position':'q_position'}, inplace = True)
driver_standings.rename(columns = {'points':'ds_points', 'position':'ds_position', 'wins':'ds_wins'}, inplace = True)
constructor_standings.rename(columns = {'points':'cs_points', 'position':'cs_position', 'wins':'cs_wins'}, inplace = True)

In [ ]:
# Merge de los datos que no requieren ser calculados a partir de otros
full = pd.merge(results, qualifying, how='inner', on=['raceId', 'driverId', 'constructorId']).merge(
    races, how='inner', on=['raceId']).merge(
    circuits, how='inner', on=['circuitId'])

Como se indicó previamente, las tablas _driver_standings_ y _constructor_standings_ contienen las variables _points_, _wins_ y _position_, que son datos obtenidos una vez finalizada la carrera correspondiente.

Como consideramos que estas son variables de mucho interés, realizamos operaciones sobre estas tablas para obtener los datos de estas columnas de la carrera inmediatamente anterior de la misma season. En resumen, la idea de estas variables es la misma pero usamos los datos al momento previo a la carrera que intentaremos predecir si un corredor estará en el podio.

Para hacerlo, agregamos las columnas date y year en driver_standings y constructor standings.

In [ ]:
# Agregamos columnas 'date' y 'year' a los dataframes según la carrera correspondiente
driver_standings = pd.merge(driver_standings, races[["raceId", "date", "year"]], on="raceId")
constructor_standings = pd.merge(constructor_standings, races[["raceId", "date", "year"]], on="raceId")

# Ordenamos los dataframes a partir de columnas 'driverId'/'constructorId' y 'date'
driver_standings = driver_standings.sort_values(by=['driverId', 'date'], ascending=True)
constructor_standings = constructor_standings.sort_values(by=["constructorId", "date"], ascending=True)

# Convertimos campo 'date' de str a datetime
driver_standings['date'] = pd.to_datetime(driver_standings['date'])
constructor_standings['date'] = pd.to_datetime(constructor_standings['date'])

# Usamos el método shift para obtener los valores anteriores (ya se encuentra ordenado)
driver_standings['ds_prev_points'] = driver_standings.groupby(['driverId', 'year'])['ds_points'].shift(1)
driver_standings['ds_prev_position'] = driver_standings.groupby(['driverId', 'year'])['ds_position'].shift(1)
driver_standings['ds_prev_wins'] = driver_standings.groupby(['driverId', 'year'])['ds_wins'].shift(1)

constructor_standings['cs_prev_points'] = constructor_standings.groupby(['constructorId', 'year'])['cs_points'].shift(1)
constructor_standings['cs_prev_position'] = constructor_standings.groupby(['constructorId', 'year'])['cs_position'].shift(1)
constructor_standings['cs_prev_wins'] = constructor_standings.groupby(['constructorId', 'year'])['cs_wins'].shift(1)

driver_standings.head()

In [ ]:
# Consulta para verificar que se haya realizado correctamente
driver_standings[(driver_standings.driverId == 1) & ((driver_standings.year == 2022) | (driver_standings.year == 2023)| (driver_standings.year == 2021))].sort_values('date', ascending=True)

In [ ]:
# Mergeamos ahora sí 'full' con 'driver_standings' y 'constructor_standings' ya modificados.
full = pd.merge(full, driver_standings[['raceId', 'driverId', 'ds_prev_points', 'ds_prev_position', 'ds_prev_wins']], how='inner', on=['raceId', 'driverId']).merge(
    constructor_standings[['raceId', 'constructorId', 'cs_prev_points', 'cs_prev_position', 'cs_prev_wins']], how='inner', on=['raceId', 'constructorId'])

# Cambiamos algunos nombres de columnas
full.rename(columns = {'ds_prev_points':'ds_points','ds_prev_position':'ds_position','ds_prev_wins':'ds_wins',
                      'cs_prev_points':'cs_points','cs_prev_position':'cs_position','cs_prev_wins':'cs_wins'}, inplace = True)

In [ ]:
# Consulta para verificar que se haya realizado correctamente
full[(full.driverId == 1) & ((full.year == 2022) | (full.year == 2023)| (full.year == 2021))].sort_values('date', ascending=True)

In [ ]:
# Creamos la columna 'is_podium' que es el target a predecir a partir de los datos que tenemos en positionOrder
full['is_podium'] = np.where(full['positionOrder'] <= 3, 1, 0)
full.drop(['positionOrder'], axis=1, inplace=True)
full.head()

In [ ]:
print('Dimensiones: ', full.shape)
print('Dimensiones sin columnas ID: ',
      full[['grid', 'q_position', 'q1', 'q2', 'q3', 'year', 'date', 'time', 'circuitRef', 'ds_points', 'ds_wins', 'ds_position',
            'cs_points', 'cs_wins', 'cs_position', 'is_podium']].shape)
full.dtypes

### 2) Análisis detallado de un conjunto de variables
#### 2.1 Balanceo de la variable de salida

In [ ]:
# Gráfico de torta para ver el balanceo de la variable de salida
full.is_podium.value_counts().plot.pie(autopct='%.2f',figsize=(6,6), labels=['No está en el podio', 'Está en el podio'], ylabel='Cant.')

En el gráfico de torta se puede observar que del total de los datos de las carreras que se encuentran en nuestro dataset, el 14.42% de los corredores se encuentran en el podio, mientras que el 85.58% no se encuentra en el mismo. 
A la hora de entrenar y medir el rendimiento del modelo, este podría realizar underfitting (no aprenderá a generalizar, podría decir que nunca hará podium para todos los casos y obtendría una exactitud del 85.58%) ya que la mayoría de los casos no se encuentra en el podio.

#### 2.2) Comportamiento de variables de entrada y su relación con la variable de salida
_Variables de entrada a analizar:_

- grid
- q_position
- q1_ms
- ds_position
- cs_position

__Posición de salida (grid)__

Analizamos la distribución de esta variable, visualizando que en la mayoría de los casos toma valores del 1 al 20. También detectamos casos en que toma el valor 0, lo que indica que el corredor comienza la carrera desde boxes (penalización).
Por otra parte, encontramos valores mayores a 20 debido a que el dataset tiene datos desde el año 1950 y desde ese momento hasta la actualidad hubo varios cambios en la cantidad de competidores.

In [ ]:
px.histogram(full, x='grid', title='Distribución posición de salida (grid)')

A partir del histograma relacionado con la variable de salida, se puede observar que cuanto más adelante comienza el vehículo, más probable que se encuentre en el podio. Los primeros datos que se encuentran en 0, es debido a que los corredores no hayan disputado la carrera, por ejemplo por rotura de auto, o porque salieron de boxes como se mencionó antes.

In [ ]:
px.histogram(full, x='grid', color='is_podium', title='Relación entre \'grid\' y \'is_podium\'', barmode='group',
            labels={"grid": "Posición de salida"})

__Posición final de clasificación (q_position)__

A partir del histograma de la variable sola, se puede observar que en la mayoría de las carreras hay 20 corredores. También nos encontramos con los casos que se superan los 20 corredores, pero menor cantidad de veces.

In [ ]:
px.histogram(full, x='q_position', title='Distribución posición final clasificación (q_position)')

En relación con la variable de salida, cuanto más lejos finalizan en la clasificación, menos probabilidades de estar en el podio  tendrán los corredores.

In [ ]:
px.histogram(full, x='q_position', color='is_podium', title='Relación entre \'q_position\' y \'is_podium\'', barmode='group',
             labels={"q_position": "Posición final clasificación"})

__Tiempo de vuelta en Q1 (q1_ms)__

Aclaración: las variables q1, q2 y q3 las transformamos a milisegundos para poder analizarlas.
A estas variables se les cambia el nombre a qn_ms.

Para la variable q1_ms, decidimos acotar los datos a un circuito en particular para poder analizarlos mejor, ya que si no hacemos lo mencionado anteriormente los tiempos de vuelta van a ser muy diferentes entre distintos circuitos y por lo tanto difíciles de analizar.

In [ ]:
# Función para convertir los valores de q1, q2 y q3 (string) en milisegundos.
def datetime_to_seconds(laptime: str) -> float:
    if laptime == '\\N' or laptime == 'nan':
        return np.nan
    time_format = "%M:%S.%f"
    dt = datetime.strptime(laptime, time_format)
    seconds = dt.minute * 60 + dt.second + dt.microsecond / 1_000_000
    return float(seconds * 1000)

In [ ]:
# Convertimos todas las columnas de tiempos de qualy a milisegundos
full['q1'] = full['q1'].astype(str)
full['q1_ms'] = np.vectorize(datetime_to_seconds)(full['q1'])

full['q2'] = full['q2'].astype(str)
full['q2_ms'] = np.vectorize(datetime_to_seconds)(full['q2'])

full['q3'] = full['q3'].astype(str)
full['q3_ms'] = np.vectorize(datetime_to_seconds)(full['q3'])

# Sacamos columnas q1, q2 y q3 ya que utilizaremos la columna pasada a milisegundos.
full.drop(['q1', 'q2', 'q3'], axis=1, inplace=True)

En el siguiente histograma, podemos ver que los tiempos de vuelta en el circuito de Suzuka van desde, aproximadamente, los 80000 a 130000 milisegundos. También se puede notar un caso extremo, en el que se registra un tiempo de vuelta cerca del millón de milisegundos. Ampliaremos sobre este dato más adelante.

In [ ]:
px.histogram(full[full.circuitRef == 'suzuka'], x='q1_ms', title='Tiempos de vuelta Q1 en Suzuka')

Relacionando los tiempos de vuelta de Q1 con la variable 'is_podium', podemos ver que los pilotos que realizan un tiempo de vuelta entre los 80 y 90 mil milisegundos tienen mayor probabilidad de entrar al podio. Se puede ver que mientras más bajo sea el tiempo, más probabilidades tendrá.

De todas formas, podemos ver un caso particular donde se realiza un tiempo entre 120 y 130 mil ms y de todas formas logra el podio.

In [ ]:
px.histogram(full[full.circuitRef == 'suzuka'], x='q1_ms', color='is_podium', title='Relación entre q1_ms y is_podium',
             barmode='group', labels={"q1_ms": "Tiempo de vuelta en Q1 (ms)"})

__Posición en campeonato de corredores (ds_position)__

En este histograma se puede observar cómo se distribuyen los datos correspondientes a las posiciones de campeonato de corredores. Se ve que en la mayoría de casos hubo hasta 20 corredores, pero en casos particulares, llegaron a correr 44 corredores distintos, como es el caso de la temporada 1994.

In [ ]:
px.histogram(full, x='ds_position', title='Distribución posiciones de corredores en campeonato')

In [ ]:
# Vemos el caso atípico mencionado anteriormente
full[full.ds_position > 20].sort_values('ds_position', ascending=False).head()

Relacionando esta variable con la variable de salida 'is_podium', nos damos cuenta la posición en el campeonato del corredor influye bastante para determinar si terminará o no dentro del podio.

Podemos destacar, que en los casos donde el corredor está primero o segundo en el campeonato, tiene más posibilidades de quedar en podio que no quedar.

In [ ]:
px.histogram(full, x='ds_position', color='is_podium', barmode='group', title='Relación entre ds_position y is_podium',
            labels={"ds_position": "Posición en campeonato"})

__Posición en campeonato de constructores (cs_position)__

En el histograma vemos que la distribución de las posiciones de los equipos en el campeonato es muy similar entre los primeros 10 puestos. Esto es así ya que en la mayoría de temporadas de F1 participan 10 equipos/constructores. Como en el caso anterior, también observamos casos que se dan con menor frecuencia, como por ejemplo la temporada de 1994, donde participaron 14 equipos.

In [ ]:
px.histogram(full, x='cs_position', title='Distribución posiciones de constructores en campeonato')

Relacionando esta variable con la variable de salida 'is_podium', nos damos cuenta la posición en el campeonato de equipos influye bastante para determinar si terminará o no dentro del podio.

Podemos destacar, que en el caso donde el equipo se encuentra primero en el campeonato, tiene más posibilidades de quedar en podio que no quedar.

In [ ]:
px.histogram(full, x='cs_position', color='is_podium', barmode='group', title='Relación entre cs_position y is_podium',
            labels={"cs_position": "Posición en campeonato"})

#### 2.3) Transformaciones sobre las variables de entrada

- cirtcuitRef: como se trata de una columna categórica, podríamos implementar One-Hot Encoder, convirtiendo la misma en muchas columnas con los diferentes nombres de referencia de los circuitos. Esto lo hacemos para evitar que se asigne un orden arbitrario a las categorías y tengan la misma importancia.

Las variables enumeradas a continuación serán escaladas. Esto se debe a que, por más que tengan un órden o jerarquía natural, los posibles valores que pueden tomar difieren entre las distintas variables, lo que puede llevar al modelo a determinar que una variable tiene más peso o importancia que otra.

Por ejemplo, la variable position puede tomar valores del 1 al 20 (en la mayoría de los casos), mientras que la variable ds_points puede tomar valores desde 0 hasta 454 (máximo valor en el dataset) o más.
Es por esto que consideramos importante escalar estas variables.

El método para escalar va a depender de los modelos que vamos a entrenar.
- grid
- position
- ds_position
- ds_wins
- ds_points
- cs_position
- cs_wins
- cs_points

Las siguientes variables aplican a lo explicado anteriormente y además decidimos pasarlas  a milisegundos para que el modelo pueda trabajar con mayor precisión sobre estos datos.
- q1_ms
- q2_ms
- q3_ms

En cuanto a las variables _'date'_ y _'time'_ se podría aplicar composición, es decir, convertir esas columnas en una sola para facilitarle el trabajo al modelo.


#### 2.4) Valores nulos y valores extremos

In [ ]:
# En el dataset se indican los valores no registrados (nulos) como '\N'. Para hacer un mejor análisis los pasamos a None o NaN
full['time'] = full['time'].replace('\\N', None)

In [ ]:
# Analisis valores nulos
full.isnull().sum()

___Valores nulos y extremos encontrados:___
- time (races): con respecto a esta variable, pudimos notar que existen varios valores nulos, es por ello que se podría aplicar algún tratamiento, como por ejemplo, considerar la hora que más se repite.
- q1_ms: en esta variable nos encontramos con un valor extremo que se ve en el gráfico a continuación. El tratamiento a aplicar sobre este dato sería eliminarlo del set de datos.
- q2_ms y q3_ms: vemos que hay valores que superan el extremo superior del gráfico de caja y bigotes, pero no los tomaremos como atípicos ya que tenemos en cuenta carreras desde 1950, donde los tiempos de vuelta no estaban muy cercanos a los de la actualidad.

Para tener en cuenta, en q1_ms, q2_ms y q3_ms tenemos valores nulos, que podemos ver que aumentan de a poco a medida que analizamos la siguiente etapa de clasificación. Esto se debe a que en F1 la clasificación va eliminando participantes entre etapas. En estos casos, tendríamos que tratarlos poniendo un valor fijo que indique que es nulo y no afecte el proceso de predicción del modelo.

In [ ]:
# Analisis valores extremos
px.box(full, y='q1_ms')

In [ ]:
px.box(full, y='q2_ms')

In [ ]:
px.box(full, y='q3_ms')

In [ ]:
# Valor extremo encontrado en los tiempos de vuelta de q1 en milisegundos
full[full.q1_ms > 1000000]

En el caso de las siguientes variables, los valores nulos son debido a que no se encontraron datos de la carrera anterior, ya que se trata de la primera carrera de la temporada. A los mismos se les podría aplicar algún tratamiento, como poner valor a mano en 0 a los puntos (points) y a las victorias (wins). Respecto a la posición (position), podríamos asignarle un valor random entre 1 y 20 que son los valores más típicos.
- ds_position
- ds_wins
- ds_points
- cs_position
- cs_wins
- cs_points

En ds_position y cs_position se pueden observar valores extremos (gráficos en punto 2.2). Estos datos no los eliminaremos porque no sabemos cómo serán organizadas las temporadas posteriores de F1, pudiendo volver a formatos anteriores donde había más corredores y equipos.

#### 2.5) Variables altamente correlacionadas con la variable "target"

In [ ]:
# Heatmap
# Armo un df nuevo quitando IDs y variables que dan error
final = full[['grid', 'q_position', 'ds_points', 'ds_position', 'ds_wins',
              'cs_points', 'cs_position', 'cs_wins', 'q1_ms', 'q2_ms', 'q3_ms', 'is_podium']]

In [ ]:
px.imshow(final.corr(), text_auto=True, width=800, height=800)

___Variables altamente correlacionadas positivamente con is_podium___
- ds_points
- ds_wins
- cs_points
- cs_wins

Estas variables tienen correlación positiva ya que como se analizó previamente, a medida que estas aumentan en valor más probabilidades tendrá de estar dentro del podio.

___Variables altamente correlacionadas negativamente con is_podium___
- grid
- q_position
- ds_position
- cs_position

Estas variables tienen correlación negativa ya que mientras más bajo es su valor, más alta será la probabilidad de finalizar en el podio. Por ejemplo, salir primero en la grilla aumenta la posibilidad de podio.

Por otra parte, las variables q1_ms, q2_ms y q3_ms tienen una correlación negativa, pero es muy cercana a 0.

### 3) Hipótesis sobre los datos

#### ___3.1) Formulación y análisis de las hipótesis___
__Hipótesis 1__

Una mejor posición de salida (grid) es más importante que la posición de clasificación (q_position) para acabar en el podio.

In [ ]:
#Hipótesis 1
f, ax = plt.subplots(1, 2, figsize=(16, 12))
ax[0].set_title('% de podios por posición de salida (grid)')
full[(full.is_podium == 1) & ((full.grid >= 1) & (full.grid <= 20))].grid.value_counts().plot.pie(ax=ax[0],autopct='%1.2f%%')
ax[1].set_title('% de podios por posición de clasificación (q_position)')
full[(full.is_podium == 1) & ((full.q_position >= 1) & (full.q_position <= 20))].q_position.value_counts().plot.pie(ax=ax[1],autopct='%1.2f%%')

A primera vista en el diagrama de torta, se podría decir que la hipótesis es verdadera, ya que existe una proporción mayor de podios en la variable grid en los primeros puestos. Pero a medida que fuimos analizando los distintos casos, los porcentajes de podios por posición de clasificación son mayores que los porcentajes de podios por posición de salida en algunos casos. 
Es por ello que la hipótesis planteada se refuta a partir de los datos, ya que no en todos los casos la posición de salida tiene más impacto que la posición de clasificación para determinar si se hará podio.

__Hipótesis 2__

Los puntos del corredor (ds_points) son más importantes que la cantidad de victorias del corredor (ds_wins) para acabar en el podio.

In [ ]:
# Hipótesis 2
px.scatter(full[(full.year >= 2012) & (full.year <= 2022)], x='ds_points', y='ds_wins', color='is_podium')

Mirando los gráficos de dispersión creados para evaluar esta hipótesis, podemos detectar que hay una relación lineal positiva entre ambas variables, por lo que en la parte superior derecha del gráfico nos encontraremos con una concentración de puntos que representan carreras en las que se termina dentro del podio.

In [ ]:
px.scatter(full[((full.year >= 2012) & (full.year <= 2022)) & (full.is_podium == 1)], x='ds_points', y='ds_wins')

Respecto a la hipótesis, podemos decir que es verdadera basándonos en los datos. Esto se debe a que si armamos un gráfico similar al anterior pero con datos que ya sabemos que finalizan dentro del podio, notamos una mayor cantidad de puntos en la parte inferior del diagrama, donde la cantidad de victorias es menor.

#### ___3.2) Hallazgos en los datos___

Antes de hacer un análisis de los datos con mayor profundidad, pensábamos que la correlación entre la posición de salida (grid) y la posición final de clasificación (q_position) iba a ser del 100%.
Sin embargo, al observar el mapa de calor, notamos que la correlación entre estas variables no es la esperada, sino de un 95%. A partir de esto, miramos los datos y nos dimos cuenta que existen diferencias debido a que un corredor puede penalizar posiciones de salida por diversos motivos, como pueden ser exceder la cantidad de cambios de componentes del monoplaza o alguna sanción obtenida durante la sesión de clasificación.
Por ejemplo, un corredor finaliza la clasificación en primera posición (q_position) pero antes de comenzar la carrera realiza un cambio de componente superando el límite y obtiene una sanción de 5 puestos, por lo que comenzará la carrera en sexta posición (grid).

También pudimos observar que en ciertas temporadas, generalmente las más antiguas, contaban con muchos más corredores que en las últimas temporadas de Formula 1.

#### Limpieza final del dataset para el entrenamiento
Realizamos la limpieza del dataset y lo exportamos en un csv para utilizarlo en el entrenamiento.

In [ ]:
# Eliminamos los IDs
full.drop(['resultId', 'raceId', 'driverId', 'constructorId', 'qualifyId', 'circuitId', ], axis=1, inplace=True)
# Ponemos la variable de salida al final del dataset
full = full[[c for c in full if c not in ['is_podium']] + ['is_podium']]
full.head()

In [ ]:
full.to_csv('f1_race_podiums.csv', index=False)